<a href="https://colab.research.google.com/github/imAdityaSatya/DocuChat/blob/main/DocuChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

QnA ChatBot version-1

In [2]:
# Install dependencies (if not already)
# !pip install pypdf transformers torch
!pip install pypdf transformers torch sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 48.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [3]:
import random
from pypdf import PdfReader
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from google.colab import files

# Greeting detection setup
GREETINGS_IN = {"hi", "hello", "hey", "good morning", "good evening", "greetings"}
GREETINGS_OUT = ["Hello!", "Hi there!", "Hey! How can I help?"]

def check_greeting(text: str) -> str:
    return random.choice(GREETINGS_OUT) if text.lower() in GREETINGS_IN else ""


In [4]:
# Functions for PDF Text Extraction & Chunking

def extract_pdf_text(path: str) -> str:
    reader = PdfReader(path)
    return "\n".join(page.extract_text() or "" for page in reader.pages)

def chunk_text(text: str, max_len: int = 400, stride: int = 50) -> list[str]:
    words = text.split()
    return [
        " ".join(words[i : i + max_len])
        for i in range(0, len(words), max_len - stride)
    ]

# Upload PDF file
uploaded = files.upload()
pdf_path = next(iter(uploaded.keys()))

# Extract and chunk text
full_text = extract_pdf_text(pdf_path)
contexts = chunk_text(full_text)
print(f"Extracted {len(contexts)} chunks from your PDF")

Saving 24. Reinforcement Learning.pdf to 24. Reinforcement Learning.pdf
Extracted 16 chunks from your PDF


In [5]:
# Load the embedding model
embedder = SentenceTransformer("all-mpnet-base-v2")

# Embed the contexts
embeddings = embedder.encode(contexts, convert_to_numpy=True, show_progress_bar=True)

# Normalize and Build FAISS Index for Retrieval
faiss.normalize_L2(embeddings)
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)
print("FAISS index built successfully")


# Initialize the QnA Pipeline
qa_pipeline = pipeline(
    "question-answering",
    model="deepset/roberta-base-squad2",
    tokenizer="deepset/roberta-base-squad2"
)
print("QnA Pipeline Initialized")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


QnA Pipeline Initialized


In [11]:
# Interactive Chatbot
print("DocuChat is ready! \nType in your question and hit Enter\tType 'exit' or 'quit' to stop\n")

# Some querry triggers
INTRO_QUERIES = {"who are you", "what's your name", "whats your name", "what can you do", "are you a bot", "are you a chatbot", "introduce yourself"}
CREATOR_QUERIES = {"who built you", "who created you", "who made you", "who's your creator", "who is your creator"}
GRATITUDE_QUERIES = {"thanks", "thank you", "good", "great", "wow", "awesome", "cool", "nice", "got it", "ok", "okay"}
GREET_QUERIES = {"how are you", "how are you doing", "how are you feeling"}

while True:
    user_input = input("You: ").strip()
    # Cleaned user input : Convert to lowercase and ignore special chars (?, !)
    cleaned_input = user_input.lower().replace("?", "").replace("!", "")

    # Exit Commands
    if cleaned_input in {"exit", "quit", "bye", "goodbye"}:
        print("Bot: Goodbye!")
        break

    # Intro query handling
    if cleaned_input in INTRO_QUERIES:
        print("Bot: I am DocuChat, a QnA chatbot that can answer questions based on any PDF you upload.")
        continue

    # Creator query handling
    if cleaned_input in CREATOR_QUERIES:
        print("Bot: I was created by Aditya Satya :) ")
        continue

    # Gratitude query handling
    if cleaned_input in GRATITUDE_QUERIES:
        print("Bot: Happy to help! \n     Let me know if there's anything else I can help you with. \n")
        continue

    # Greet query handling
    if cleaned_input in GREET_QUERIES:
        print("Bot: I'm doing great, thanks for asking. \n     How may I help you?\n")
        continue

    # Handle greetings
    greet = check_greeting(cleaned_input)
    if greet:
        print(f"Bot: {greet}")
        continue

    # Handle the empty input
    if not user_input:
        print("Bot: Your input is empty. Please ask a question.")
        continue


    # Encode query and retrieve top-k contexts
    q_emb = embedder.encode(user_input, convert_to_numpy=True)
    faiss.normalize_L2(q_emb.reshape(1, -1))
    D, I = index.search(q_emb.reshape(1, -1), k=3)  # retrieve top-3

    # Run QA on retrieved chunks
    best = {"score": 0.0, "answer": "Sorry, this is beyond the scope of abilities."}
    for idx in I[0]:
        result = qa_pipeline(question=user_input, context=contexts[idx])
        if result["score"] > best["score"]:
            best = result

    # Confidence threshold
    if best["score"] < 0.25:
        print("Bot: Sorry, I'm not really sure about that.")
    else:
        print(f"Bot: {best['answer']}")


DocuChat is ready! 
Type in your question and hit Enter	Type 'exit' or 'quit' to stop

You: hi
Bot: Hello!
You: how are you?
Bot: I'm doing great, thanks for asking. 
     How may I help you?

You: how are you!
Bot: I'm doing great, thanks for asking. 
     How may I help you?

You: whoa re you?!?!?!
Bot: Sorry, I'm not really sure about that.
You: who are you?!?!?!
Bot: I am DocuChat, a QnA chatbot that can answer questions based on any PDF you upload.
You: great
Bot: Happy to help! 
     Let me know if there's anything else I can help you with. 

You: bye
Bot: Goodbye!
